# Regression example with erros Monte-Carlo

In [ ]:
%matplotlib qt
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [ ]:
import sys
sys.path.append("../")
import numpy as np
from numpy.linalg import norm
from matplotlib import pyplot as plt
from tqdm import tqdm
from mmhuber.mmalgorithm import hubreg
from mmhuber.least_squares import estimate_beta_sigma
from sklearn.linear_model import HuberRegressor
from sklearn.datasets import make_regression

In [ ]:
# Setting the simulation parameters
n_samples = 500
n_features = 250
n_trials = 240
c = 1.345
SNR_db = 20
sigma = 20
err_prob_vec = np.arange(0,0.10,0.01)

In [ ]:
error_beta_ls = []
error_sigma_ls = []
error_beta_mm_huber = []
error_sigma_mm_huber = []
error_beta_sklearn_huber = []
error_sigma_sklearn_huber = []
for err_prob in tqdm(err_prob_vec):

    error_beta_ls_temp = 0
    error_sigma_ls_temp = 0
    error_beta_mm_huber_temp = 0
    error_sigma_mm_huber_temp = 0
    error_beta_sklearn_huber_temp = 0
    error_sigma_sklearn_huber_temp = 0
    for trial in range(n_trials):
        # Generating data
        X, y, beta = make_regression(n_samples=n_samples,    
              n_features=n_features, n_informative=n_features, coef=True,  
              random_state=trial)
        e0 = np.random.randn(n_samples)
        # con = (norm(X@beta) / norm(e0))**2
        # sigma = np.sqrt(con*10*10**(-SNR_db/10))
        y +=  sigma*e0

        # Flipping some data to account as errors
        if err_prob > 0:
            indexes = np.random.binomial(1, err_prob, n_samples)
            y[indexes] = -y[indexes]

        # LS estimate
        beta_estimate, sigma_estimate = estimate_beta_sigma(X, y)
        error_beta_ls_temp += norm(beta - beta_estimate, ord=2)**2 / norm(beta, ord=2)**2 
        error_sigma_ls_temp += (sigma_estimate)**2 / (sigma**2) 

        # MM-huber estimate
        beta_estimate, sigma_estimate = hubreg(y, X, c, beta_0='LS', sigma_0='LS', mu='optimal', lbda='optimal', 
                             check_decreasing=True, n_iter=100, pbar=False,
                             epsilon=1e-5)
        error_beta_mm_huber_temp += norm(beta - beta_estimate, ord=2)**2 / norm(beta, ord=2)**2 
        error_sigma_mm_huber_temp += (sigma_estimate)**2 / (sigma**2)

        # Sklearn huber
        huber = HuberRegressor(epsilon=c).fit(X, y)
        beta_estimate = huber.coef_
        sigma_estimate = huber.scale_
        error_beta_sklearn_huber_temp += norm(beta - beta_estimate, ord=2)**2 / norm(beta, ord=2)**2 
        error_sigma_sklearn_huber_temp += (sigma_estimate)**2 / (sigma**2)


    error_beta_ls.append( error_beta_ls_temp / n_trials )
    error_sigma_ls.append( error_sigma_ls_temp / n_trials )
    error_beta_mm_huber.append( error_beta_mm_huber_temp / n_trials )
    error_sigma_mm_huber.append( error_sigma_mm_huber_temp / n_trials )
    error_beta_sklearn_huber.append( error_beta_sklearn_huber_temp / n_trials )
    error_sigma_sklearn_huber.append( error_sigma_sklearn_huber_temp / n_trials )



In [ ]:
plt.figure(figsize=(5,4))
plt.plot(err_prob_vec, error_beta_ls, marker='o', label='LS')
plt.plot(err_prob_vec, error_beta_mm_huber, marker='s', label='MM-Huber')
plt.plot(err_prob_vec, error_beta_sklearn_huber, marker='d', label='Sklearn-Huber')
plt.legend()
plt.xlabel('$\epsilon$')
plt.ylabel('$\| \hat{\\beta} - \\beta \|^2_2/\| \\beta \|^2_2$')

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(err_prob_vec, np.log10(error_sigma_ls), marker='o', label='LS')
plt.plot(err_prob_vec, np.log10(error_sigma_mm_huber), marker='s', label='MM-Huber')
plt.plot(err_prob_vec, np.log10(error_sigma_sklearn_huber), marker='d', label='Sklearn-Huber')
plt.legend()
plt.xlabel('$\epsilon$')
plt.ylabel('$ \log_{10}^{2}(  \hat{\\sigma}/\\sigma  )$')

In [ ]:
X.shape

In [ ]:
y.shape